# 面向过程的有限元编程

我们首先将前几节的代码汇总一下，并将其分为三个部分：有限元前处理、有限元求解器和有限元后处理。

有限元前处理（由用户输入）：

In [1]:
import numpy as np
import pygmsh

# 定义网格
with pygmsh.geo.Geometry() as geom:
    geom.add_polygon([[0.0, 0.0],[1.0, 0.0],[1.0, 1.0],[0.0, 1.0],],mesh_size=0.5,)
    mesh = geom.generate_mesh()
nodes=mesh.points
elements=mesh.cells_dict['triangle']

# 定义材料
E=10.0
nv=0.3

# 定义载荷
x_fix={0:0.0,2:0.0,3:0.0}
y_fix={0:0.0,2:0.0,3:0.0}
f_given={5:(1.0,0.0)}

有限元求解器：

In [2]:
# 第一步
# 材料本构：刚度矩阵（和切向刚度矩阵）
D=E/((1.0+nv)*(1.0-2*nv))*np.array([
    [1.0-nv,nv,0.0], 
    [nv,1.0-nv,0.0], 
    [0.0,0.0,0.5-nv]])

# 第二步
# 生成总体系统：节点位移、节点力和刚度矩阵
# 总体系统大小
len_global=2*len(nodes)
deform=np.zeros(len_global)
force=np.zeros(len_global)
K=np.zeros((len_global,len_global))
#初始化节点位移
for node in x_fix:
    deform[2*node]=x_fix[node]
for node in y_fix:
    deform[2*node+1]=y_fix[node]
#初始化节点力
for node in f_given:
    force[2*node]=f_given[node][0]
    force[2*node+1]=f_given[node][1]
#初始化刚度矩阵
for element in elements:
    x0=nodes[element[0]][0]
    y0=nodes[element[0]][1]
    x1=nodes[element[1]][0]
    y1=nodes[element[1]][1]
    x2=nodes[element[2]][0]
    y2=nodes[element[2]][1]
    area_2=np.linalg.det(np.array([
        [1.0,x0,y0], 
        [1.0,x1,y1], 
        [1.0,x2,y2]]))
    B=1/area_2*np.array([
        [y1-y2,0.0,y2-y0,0.0,y0-y1,0.0], 
        [0.0,x2-x1,0.0,x0-x2,0.0,x1-x0], 
        [x2-x1,y1-y2,x0-x2,y2-y0,x1-x0,y0-y1]])
    K_element=area_2*B.T@D@B
    deform_global_index=np.array([[2*element[i],2*element[i]+1] for i in range(3)],dtype=np.uint64).reshape(-1)
    for i_local,i_global in enumerate(deform_global_index):
        for j_local,j_global in enumerate(deform_global_index):
            K[i_global,j_global]+=K_element[i_local,j_local]

# 第三步
# 生成约化系统：节点位移、节点力和刚度矩阵
# 生成约化系统指标列表
deform_free_index=[]
for node,_ in enumerate(nodes):
    if node not in x_fix:
        deform_free_index.append(2*node)
    if node not in y_fix:
        deform_free_index.append(2*node+1)
# 约化系统大小
len_reduce=len(deform_free_index)
deform_reduce=np.empty(len_reduce)
force_reduce=np.zeros(len_reduce)
K_reduce=np.zeros((len_reduce,len_reduce))
# 约化节点力
for i_reduce,i_global in enumerate(deform_free_index):
    force_reduce[i_reduce]=force[i_global]
# 约化刚度矩阵
for i_reduce,i_global in enumerate(deform_free_index):
    for j_reduce,j_global in enumerate(deform_free_index):
        K_reduce[i_reduce,j_reduce]=K[i_global,j_global]

# 第四步
# 求解约化系统
deform_reduce=np.linalg.solve(K_reduce,force_reduce)

# 第五步
# 更新总体系统：节点位移和节点力
# 更新节点位移
for i_reduce,i_global in enumerate(deform_free_index):
    deform[i_global]=deform_reduce[i_reduce]
# 更新节点力
force=K@deform

有限元后处理（在面向对象版本中详述）：

In [3]:
# 展示计算结果
# 画图、获取数据
# 举一例，展示节点位移
print(deform.reshape(len(nodes),2))

[[ 0.          0.        ]
 [ 0.03286534 -0.00350798]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.03103911  0.00828099]
 [ 0.07988051  0.00308401]
 [ 0.01730777 -0.01141488]
 [ 0.03388964 -0.00244238]
 [ 0.0269345  -0.00348095]
 [ 0.0341835  -0.00432742]
 [ 0.04225387 -0.01213036]
 [ 0.05115491  0.00442899]]
